In [2]:
# ================================================
# ИМПОРТ БИБЛИОТЕК
# ================================================
 
import pandas as pd   # основная библиотека для работы с таблицами
import numpy as np    # библиотека для численных вычислений
 
# Эти две библиотеки не обязательны, но полезны для визуализации выбросов
import matplotlib.pyplot as plt
import seaborn as sns
 
 
# ================================================
# НАСТРОЙКИ: КАКИЕ СТОЛБЦЫ БУДЕМ ИЗУЧАТЬ
# ================================================
 
# Здесь мы явно указываем, какие 8 колонок берём для анализа.
# ❗ ЕСЛИ у тебя в df другие названия колонок,
#    просто поменяй элементы списков ниже.
 
numeric_cols = [
    "rectal_temp",         # ректальная температура (число)
    "pulse",               # пульс (число)
    "respiratory_rate",    # частота дыхания (число)
    "packed_cell_volume"   # гематокрит (число)
]
 
categorical_cols = [
    "pain",                # уровень боли (категориальная)
    "abdominal_distension",# вздутие живота (категориальная)
    "outcome",             # исход (категориальная)
    "surgery"              # была ли операция (категориальная)
]
 
selected_cols = numeric_cols + categorical_cols
 
 
# ================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ================================================
 
def select_columns(df: pd.DataFrame,
                   cols: list[str]) -> pd.DataFrame:
    """
    Функция выбирает из исходного датафрейма только нужные столбцы.
 
    df   - исходный датафрейм со всеми данными
    cols - список названий столбцов, которые нужны для анализа
 
    Возвращает новый датафрейм только с этими столбцами.
    """
    # .copy() — делаем копию, чтобы не портить исходный df
    df_sel = df[cols].copy()
    return df_sel
 
 
def basic_stats_numeric(df: pd.DataFrame,
                        numeric_cols: list[str]) -> pd.DataFrame:
    """
    Считает базовые статистики для ЧИСЛОВЫХ столбцов.
 
    Для каждого столбца получаем:
    - count (сколько наблюдений, не NaN)
    - mean (среднее)
    - std (стандартное отклонение)
    - min, max
    - 25%, 50% (медиана), 75% квартили
    - моду (mode)
 
    Возвращает датафрейм с описанием (describe),
    а моду печатает отдельно.
    """
    print("=== Базовые статистики для числовых столбцов ===\n")
 
    # describe() сразу считает основные статистики для всех числовых колонок
    desc = df[numeric_cols].describe().T  # .T — транспонируем, чтобы столбцы были строками
    print(desc, "\n")
 
    # Считаем моду (наиболее часто встречающееся значение) по каждому столбцу
    print("Моды для числовых столбцов:")
    for col in numeric_cols:
        mode_series = df[col].mode(dropna=True)
        if mode_series.empty:
            print(f"  {col}: мода не определена (нет данных)")
        else:
            # mode() может вернуть несколько значений, возьмём первое
            print(f"  {col}: мода = {mode_series.iloc[0]}")
    print()
 
    return desc
 
 
def basic_stats_categorical(df: pd.DataFrame,
                            categorical_cols: list[str]) -> None:
    """
    Считает базовые статистики для КАТЕГОРИАЛЬНЫХ столбцов.
 
    Для каждой категории:
    - выводим количество различных значений (nunique)
    - выводим частоты значений (value_counts)
    - находим моду (самое частое значение)
    """
    print("=== Базовые статистики для категориальных столбцов ===\n")
 
    for col in categorical_cols:
        print(f"Столбец: {col}")
        print(f"  Количество уникальных значений: {df[col].nunique(dropna=True)}")
 
        # value_counts показывает, сколько раз встречается каждое значение
        print("  Частоты значений (включая NaN):")
        print(df[col].value_counts(dropna=False))
 
        # Мода
        mode_series = df[col].mode(dropna=True)
        if mode_series.empty:
            print("  Мода: отсутствует (нет данных)")
        else:
            print(f"  Мода: {mode_series.iloc[0]}")
        print("-" * 40)
 
 
def find_outliers_iqr(series: pd.Series) -> tuple[pd.Series, float, float]:
    """
    Ищем выбросы в одном числовом столбце с помощью правила IQR.
 
    IQR (interquartile range) = Q3 - Q1
    Нижняя граница = Q1 - 1.5 * IQR
    Верхняя граница = Q3 + 1.5 * IQR
 
    Всё, что < нижней границы или > верхней границы, считаем выбросами.
 
    Возвращаем:
    - булеву маску (True, где выброс),
    - нижнюю границу,
    - верхнюю границу.
    """
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
 
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
 
    # mask = True там, где значение < lower_bound или > upper_bound
    mask = (series < lower_bound) | (series > upper_bound)
 
    return mask, lower_bound, upper_bound
 
 
def analyze_outliers(df: pd.DataFrame,
                     numeric_cols: list[str]) -> pd.DataFrame:
    """
    Для каждого числового столбца:
    - находим выбросы по правилу IQR
    - считаем их количество
    - выводим границы
 
    Возвращаем датафрейм с суммарной информацией по выбросам:
    столбец, количество выбросов, доля выбросов.
    """
    print("=== Анализ выбросов (IQR-метод) для числовых столбцов ===\n")
 
    rows = []
 
    for col in numeric_cols:
        series = df[col].dropna()  # игнорируем пропуски при поиске выбросов
        mask, lower, upper = find_outliers_iqr(series)
 
        n_outliers = mask.sum()  # сколько True
        n_total = series.shape[0]
        share = n_outliers / n_total if n_total > 0 else 0
 
        print(f"Столбец: {col}")
        print(f"  Нижняя граница: {lower:.3f}")
        print(f"  Верхняя граница: {upper:.3f}")
        print(f"  Количество выбросов: {n_outliers} из {n_total} ({share:.2%})")
 
        # Можно при желании посмотреть сами выбросы:
        # print(series[mask])
 
        print("-" * 40)
 
        rows.append({
            "column": col,
            "n_outliers": n_outliers,
            "n_non_null": n_total,
            "share_outliers": share
        })
 
    # Делаем из списка словарей итоговый датафрейм
    outliers_summary = pd.DataFrame(rows)
    return outliers_summary
 
 
def plot_outliers_boxplots(df: pd.DataFrame,
                           numeric_cols: list[str]) -> None:
    """
    Дополнительная функция: рисуем boxplot'ы для числовых столбцов,
    чтобы визуально увидеть выбросы.
 
    Это НЕ обязательно для ДЗ, но полезно для понимания.
    """
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
 
    # Перестраиваем данные в "длинный" формат для удобства рисования
    df_long = df[numeric_cols].melt(var_name="column", value_name="value")
 
    sns.boxplot(data=df_long, x="column", y="value")
    plt.title("Boxplot для числовых столбцов (выбросы видны как точки вне усов)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
 
 
def missing_values_report(df: pd.DataFrame,
                          cols: list[str]) -> pd.DataFrame:
    """
    Считает количество пропусков (NaN) по выбранным столбцам.
 
    Возвращает датафрейм с:
    - column      — имя столбца
    - n_missing   — сколько пропусков
    - share_missing — доля пропусков
    """
    print("=== Отчёт по пропускам (NaN) ===\n")
 
    rows = []
 
    for col in cols:
        n_missing = df[col].isna().sum()
        n_total = df[col].shape[0]
        share = n_missing / n_total if n_total > 0 else 0
 
        print(f"{col}: пропусков {n_missing} из {n_total} ({share:.2%})")
 
        rows.append({
            "column": col,
            "n_missing": n_missing,
            "n_total": n_total,
            "share_missing": share
        })
 
    print()
    return pd.DataFrame(rows)
 
 
def fill_missing_values(df: pd.DataFrame,
                        numeric_cols: list[str],
                        categorical_cols: list[str]) -> pd.DataFrame:
    """
    Создаёт НОВЫЙ датафрейм, где НЕТ пропусков.
 
    Принята следующая стратегия (её потом можно описать в отчёте):
    - для числовых столбцов: заполняем пропуски МЕДИАНОЙ
      (устойчива к выбросам, хороший дефолтный выбор)
    - для категориальных столбцов: заполняем пропуски МОДОЙ
      (самая частая категория)
 
    Возвращает df_filled — копию df без пропусков.
    """
    df_filled = df.copy()
 
    # Числовые
    for col in numeric_cols:
        median_value = df_filled[col].median()
        print(f"Заполняем пропуски в числовом столбце {col} медианой = {median_value}")
        df_filled[col] = df_filled[col].fillna(median_value)
 
    # Категориальные
    for col in categorical_cols:
        mode_series = df_filled[col].mode(dropna=True)
        if mode_series.empty:
            # если вдруг все значения NaN — можно придумать спец. категорию
            fill_value = "Unknown"
            print(f"Все значения в {col} пропущены, заполняем 'Unknown'")
        else:
            fill_value = mode_series.iloc[0]
            print(f"Заполняем пропуски в категориальном столбце {col} модой = {fill_value}")
 
        df_filled[col] = df_filled[col].fillna(fill_value)
 
    print("\nВсе пропуски заполнены.")
    return df_filled
 
 
# ================================================
# ОСНОВНОЙ "ОРКЕСТРАТОР" ДЛЯ ДЗ
# ================================================
 
def run_homework_analysis(df: pd.DataFrame) -> None:
    """
    Основная функция, которая запускает все шаги домашнего задания:
 
    1) Базовое изучение: выбор 8 колонок, расчёт базовых метрик.
    2) Работа с выбросами: поиск выбросов в числовых столбцах.
    3) Работа с пропусками: подсчёт пропусков, заполнение и получение df без NaN.
 
    df — датафрейм со всеми исходными данными (из ноутбука).
    """
 
    # ---------- Задание 1. Базовое изучение ----------
    print("\n========== ЗАДАНИЕ 1: БАЗОВОЕ ИЗУЧЕНИЕ ==========\n")
 
    df_sel = select_columns(df, selected_cols)
 
    # Базовые статистики для числовых
    desc_numeric = basic_stats_numeric(df_sel, numeric_cols)
 
    # Базовые статистики для категориальных
    basic_stats_categorical(df_sel, categorical_cols)
 
    # Здесь, в отдельной markdown-ячейке в ноутбуке,
    # можно кратко описать результаты: диапазоны, медианы,
    # какие значения категорий чаще встречаются и т.д.
 
    # ---------- Задание 2. Работа с выбросами ----------
    print("\n========== ЗАДАНИЕ 2: РАБОТА С ВЫБРОСАМИ ==========\n")
 
    outliers_summary = analyze_outliers(df_sel, numeric_cols)
 
    print("\nСводная таблица по выбросам:")
    print(outliers_summary)
 
    # Дополнительно можно визуализировать выбросы
    # (по желанию, не обязательно для сдачи ДЗ):
    # plot_outliers_boxplots(df_sel, numeric_cols)
 
    # В отдельной markdown-ячейке:
    # - сформулировать гипотезы, откуда взялись выбросы
    #   (ошибки ввода, редкие тяжёлые состояния и т.п.)
    # - принять решение: оставляем, обрезаем, заменяем и т.д.
    # В этом коде мы НИЧЕГО с выбросами не делаем автоматически,
    # только считаем и показываем.
 
    # ---------- Задание 3. Работа с пропусками ----------
    print("\n========== ЗАДАНИЕ 3: РАБОТА С ПРОПУСКАМИ ==========\n")
 
    # 1) Считаем пропуски
    missing_report = missing_values_report(df_sel, selected_cols)
    print("Таблица с пропусками:")
    print(missing_report, "\n")
 
    # 2) Заполняем пропуски по выбранной стратегии (медиана / мода)
    df_no_missing = fill_missing_values(df_sel, numeric_cols, categorical_cols)
 
    # Проверим, что пропусков действительно нет
    print("\nПроверяем, остались ли пропуски:")
    print(df_no_missing.isna().sum())
 
    # Теперь df_no_missing — датафрейм БЕЗ пропусков,
    # его можно использовать дальше для моделей и т.п.
    # При желании можно вернуть его из функции:
    # return df_no_missing
 
 
# ================================================
# ПРИМЕР ЗАПУСКА В НОУТБУКЕ
# ================================================
 
# Предполагаем, что в Базовые_Понятния.ipynb у тебя уже есть df.
# Тогда достаточно вызвать:
#
# run_homework_analysis(df)
#
# Если df пока нет, можно загрузить свои данные, например так:
#
df = pd.read_csv("horse-colic.csv")  # тут заменить на свой путь и параметры
run_homework_analysis(df)

FileNotFoundError: [Errno 2] No such file or directory: 'horse-colic.csv'